<a href="https://colab.research.google.com/github/Hackathorn/WILS-MNIST/blob/main/notebooks/Nepture_Pytorch_Support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip uninstall neptune-client
!pip install neptune

In [2]:
import neptune
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from neptune.types import File
from neptune.utils import stringify_unsupported
from torchvision import datasets, transforms


In [3]:
# Step 1: Initialize Neptune and create new Neptune run
run = neptune.init_run(
    project="hack-richard/WILS-MNIST-Neptune-Pytorch-Support",
    tags="Using MNIST",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vbmV3LXVpLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9uZXctdWkubmVwdHVuZS5haSIsImFwaV9rZXkiOiIzMjAxNTIyOC01NWJmLTRmYmYtYmUzMC1mZTFlMWE1MGVhODgifQ==",
    source_files=["*.py"],
)


<ipython-input-3-f02ccd2dd318>:2: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(


https://new-ui.neptune.ai/hack-richard/WILS-MNIST-Neptune-Pytorch-Support/e/WILS-5


In [4]:
params = {
    "lr": 1e-2,
    "batch_size": 128,
    "input_sz": 28 * 28 * 1,        # 32 * 32 * 3, 
    "n_classes": 10,
    "model_filename": "basemodel",
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}


In [5]:
# Model & Dataset
class BaseModel(nn.Module):
    def __init__(self, input_sz, hidden_dim, n_classes):
        super(BaseModel, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_sz, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, n_classes),
        )

    def forward(self, input):
        x = input.view(-1, 28*28*1)
        # x = input.view(-1, 32 * 32 * 3)
        return self.main(x)




## Use MNIST instead of CIFAR10

Derived from https://gist.github.com/xmfbit/b27cdbff68870418bdb8cefa86a2d558

In [6]:
data_dir = 'data/MNIST'
# if not os.path.exists(root):
#     os.mkdir(root)

data_tfms = transforms.Compose([transforms.ToTensor(), 
                            transforms.Normalize((0.5,), (1.0,))],
                           )
# if not exist, download mnist dataset
train_set = datasets.MNIST(data_dir, train=True, transform=data_tfms, download=True)
test_set = datasets.MNIST(data_dir, train=False, transform=data_tfms, download=True)
combine_set = torch.utils.data.ConcatDataset([train_set, test_set])

batch_size = 1      # was 100

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)
combine_loader = torch.utils.data.DataLoader(
                dataset=combine_set,
                batch_size=batch_size,
                shuffle=True)

dataset_size = {"train": len(train_set), 
                "test": len(test_set),
                "combine": len(combine_set),
                }

## Load CIFAR10 dataset

In [ ]:
# # Experiment Config
# data_dir = "data/CIFAR10"
# compressed_ds = "./data/CIFAR10/cifar-10-python.tar.gz"
# data_tfms = {
#     "train": transforms.Compose(
#         [
#             transforms.RandomHorizontalFlip(),
#             transforms.ToTensor(),
#             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#         ]
#     ),
#     "val": transforms.Compose(
#     [
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#     ]
#     ),
# }


In [13]:
# trainset = datasets.CIFAR10(data_dir, transform=data_tfms["train"], download=True)
# train_loader = torch.utils.data.DataLoader(
#     trainset, batch_size=params["batch_size"], shuffle=True, num_workers=0
# )
# dataset_size = {"train": len(trainset)}

# validset = datasets.CIFAR10(data_dir, train=False, transform=data_tfms["train"], download=True)
# valid_loader = torch.utils.data.DataLoader(validset, batch_size=params["batch_size"])
# dataset_size = {"train": len(trainset), "val": len(validset)}

Files already downloaded and verified


In [7]:
# Instatiate model, criterion and optimizer
model = BaseModel(params["input_sz"], params["input_sz"], params["n_classes"]).to(params["device"])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=params["lr"])


In [8]:
classes = [0,1,2,3,4,5,6,7,8,9]

# classes = [
#     "airplane",
#     "automobile",
#     "bird",
#     "cat",
#     "deer",
#     "dog",
#     "frog",
#     "horse",
#     "ship",
#     "truck",
# ]

In [9]:
# Step 2: Log config & pararameters
run["config/dataset/path"] = data_dir
run["config/dataset/transforms"] = stringify_unsupported(data_tfms)
run["config/dataset/size"] = dataset_size
run["config/model"] = type(model).__name__
run["config/criterion"] = type(criterion).__name__
run["config/optimizer"] = type(optimizer).__name__
run["config/hyperparameters"] = stringify_unsupported(params)
run["config/classes"] = stringify_unsupported(classes)


In [11]:
# Step 3: Log losses and metrics
for i, (x, y) in enumerate(train_loader, 0):
    x, y = x.to(params["device"]), y.to(params["device"])
    optimizer.zero_grad()
    # outputs = model.forward(x, params["input_sz"])
    outputs = model.forward(x)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, y)
    acc = (torch.sum(preds == y.data)) / len(x)

    # Log batch loss
    run["training/batch/loss"].append(loss)

    # Log batch accuracy
    run["training/batch/acc"].append(acc)

    loss.backward()
    optimizer.step()


In [ ]:
# More options


In [20]:
# Step 4: Saving model
fname = params["model_filename"]

# Saving model architecture to .txt
with open(f"./{fname}_arch.txt", "w") as f:
    f.write(str(model))
# Saving model weights .pth
torch.save(model.state_dict(), f"./{fname}.pth")    #TODO where is this save



In [21]:
# Step 4.1: Log model archictecture & weights
run[f"io_files/artifacts/{params['model_filename']}_arch"].upload(
    f"./{params['model_filename']}_arch.txt"
)
run[f"io_files/artifacts/{params['model_filename']}"].upload(f"./{params['model_filename']}.pth")



In [22]:
# Step 5: Log Torch Tensors as images with predictions

# Getting batch
dataiter = iter(test_loader)
images, labels = next(dataiter)
model.eval()

# Moving model to cpu for inference
if torch.cuda.is_available():
    model.to("cpu")

# Predict batch of n_samples
n_samples = 50
imgs = images[:n_samples]
probs = F.softmax(model(imgs), dim=1)

# Decode probs and Log tensors as image
for i, ps in enumerate(probs):
    pred = classes[torch.argmax(ps)]
    ground_truth = classes[labels[i]]
    description = "\n".join(
        [f"class {classes[n]}: {np.round(p.detach().numpy() * 100, 2)}%" for n, p in enumerate(ps)]
    )

    # Log Series of Tensors as Image and Predictions.
    run["images/predictions"].append(
        File.as_image(imgs[i].squeeze().permute(2, 1, 0).clip(0, 1)),
        name=f"{i}_{pred}_{ground_truth}",
        description=description,
    )